# 시카고 샌드위치 맛집 분석

In [46]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, urljoin

###### * 전체 html 코드 받기

In [9]:
url = 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'

html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

soup

In [11]:
# 'div'태그의 class가 'sammy'인 것을 출력
print(soup.find_all('div', 'sammy'))

[<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br>
Old Oak Tap<br>
<em>Read more</em> </br></br></a></div>
</div>, <div class="sammy" style="position: relative;">
<div class="sammyRank">2</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/"><b>Fried Bologna</b><br/>
Au Cheval<br/>
<em>Read more</em> </a></div>
</div>, <div class="sammy" style="position: relative;">
<div class="sammyRank">3</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/"><b>Woodland Mushroom</b><br/>
Xoco<br/>
<em>Read more</em> </a></div>
</div>, <div class="sammy" style="position: relative;">
<div class="sammyRank">4</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Al

In [13]:
# url에 있는 맛집의 수가 50개이므로 50이 나온다.
len(soup.find_all('div', 'sammy'))

50

In [18]:
# 맛집 rank 
print(soup.find_all('div', 'sammy')[0])

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br>
Old Oak Tap<br>
<em>Read more</em> </br></br></a></div>
</div>


### * 원하는 데이터 추출하고 정리하기

In [28]:
tmp_one = soup.find_all('div', 'sammy')[0]
type(tmp_one)
# bs4.element.Tag

bs4.element.Tag

In [29]:
tmp_one.find(class_='sammyRank')
# <div class="sammyRank">1</div>

<div class="sammyRank">1</div>

###### * 랭킹 값 추출

In [37]:
tmp_one.find(class_='sammyRank').get_text()
# '1'

'1'

###### * 메뉴 이름과 가게 이름 추출(BLT, Old Oak Tap)

In [38]:
tmp_one.find(class_='sammyListing').get_text()
# 'BLT\r\nOld Oak Tap\nRead more '

'BLT\r\nOld Oak Tap\nRead more '

In [36]:
# a태그에서 href정보로 클릭했을 때 연결될 주소
tmp_one.find('a')['href']

'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

### * Regular Express(정규식)

In [44]:
import re # 정규식

tmp_string = tmp_one.find(class_='sammyListing').get_text()
# 'BLT\r\nOld Oak Tap\nRead more '

re.split(('\n|\r\n'), tmp_string) # \n이거나 \r\n이면 분리시키겠다는 의미

print(re.split(('\n|\r\n'), tmp_string)[0])
print(re.split(('\n|\r\n'), tmp_string)[1])

BLT
Old Oak Tap


###### * 메뉴 명과 가게 이름을 출력한다.

In [56]:
rank = []
main_menu = []
cafe_name = []
url_add = []

list_soup = soup.find_all('div', 'sammy')

for item in list_soup:
    rank.append(item.find(class_='sammyRank').get_text())
    # 순위 append
    
    tmp_string = item.find(class_='sammyListing').get_text()
    
    main_menu.append(re.split(('\n|\r\n'), tmp_string)[0])
    # 메뉴 명 append
    cafe_name.append(re.split(('\n|\r\n'), tmp_string)[1])
    # 가게 이름 append
    
    url_add.append(urljoin(url_base, item.find('a')['href']))
    # url append

In [58]:
rank
main_menu
cafe_name
url_add

['http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Hendrickx-Belgian-Bread-Crafter-Belgian-Chicken-Curry-Salad/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Acadia-Lobster-Roll/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Birchwood-Kitchen-Smoked-Salmon-Salad/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/

###### * 순위, 메뉴, 가게, URL 추출한 내용을 데이터프레임에 정리

In [60]:
import pandas as pd

data = {'Rank':rank, 'Menu':main_menu, 'Cafe':cafe_name, 'URL':url_add}
df = pd.DataFrame(data)
df

,Rank,Menu,Cafe,URL
0,1,BLT,Old Oak Tap,http://www.chicagomag.com/Chicago-Magazine/Nov...
1,2,Fried Bologna,Au Cheval,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,3,Woodland Mushroom,Xoco,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,4,Roast Beef,Al’s Deli,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,5,PB&L,Publican Quality Meats,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,6,Belgian Chicken Curry Salad,Hendrickx Belgian Bread Crafter,http://www.chicagomag.com/Chicago-Magazine/Nov...
6,7,Lobster Roll,Acadia,http://www.chicagomag.com/Chicago-Magazine/Nov...
7,8,Smoked Salmon Salad,Birchwood Kitchen,http://www.chicagomag.com/Chicago-Magazine/Nov...
8,9,Atomica Cemitas,Cemitas Puebla,http://www.chicagomag.com/Chicago-Magazine/Nov...
9,10,Grilled Laughing Bird Shrimp and Fried Po’ Boy,Nana,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [62]:
# columns 순서 재배치
df = pd.DataFrame(data, columns=['Rank','Cafe','Menu','URL'])
df

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...
1,2,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,3,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,4,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,5,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,6,Hendrickx Belgian Bread Crafter,Belgian Chicken Curry Salad,http://www.chicagomag.com/Chicago-Magazine/Nov...
6,7,Acadia,Lobster Roll,http://www.chicagomag.com/Chicago-Magazine/Nov...
7,8,Birchwood Kitchen,Smoked Salmon Salad,http://www.chicagomag.com/Chicago-Magazine/Nov...
8,9,Cemitas Puebla,Atomica Cemitas,http://www.chicagomag.com/Chicago-Magazine/Nov...
9,10,Nana,Grilled Laughing Bird Shrimp and Fried Po’ Boy,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [63]:
df.to_csv('best_sandwiches_list_chicago.csv', sep=',', encoding='utf-8')